In [10]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torchaudio

# Load the fine-tuned Whisper model and processor
#model = WhisperForConditionalGeneration.from_pretrained('fine-tuned-whisper-model')
#processor = WhisperProcessor.from_pretrained('fine-tuned-whisper-processor')

WHISPER_MODEL_NAME = 'openai/whisper-large-v3-turbo'
WHISPER_PROCESSOR_NAME = 'openai/whisper-large-v3-turbo'

model = WhisperForConditionalGeneration.from_pretrained(WHISPER_MODEL_NAME)
processor = WhisperProcessor.from_pretrained(WHISPER_PROCESSOR_NAME)

# Set the model to evaluation mode and move it to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# Function to load and preprocess audio for inference
def load_audio(file_path):
    # Load the audio file
    waveform, sample_rate = torchaudio.load(file_path)

    # Resample the audio to the target sample rate if necessary
    target_sample_rate = 16000  # Whisper models expect a sample rate of 16kHz
    if sample_rate != target_sample_rate:
        waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)(waveform)
    
    return waveform.squeeze(0).numpy()

# Inference function 
def transcribe_audio(file_path):
    # Load and preprocess the audio
    audio_array = load_audio(file_path)
    
    # Process the audio in chunks of 30 seconds
    chunk_size = 30 * 16000  # 30 seconds in samples
    transcription = ""
    
    for i in range(0, len(audio_array), chunk_size):
        chunk = audio_array[i:i + chunk_size]
        
        # Process the chunk using the WhisperProcessor
        input_features = processor(chunk, return_tensors="pt", sampling_rate=16000).input_features
        
        # Move the input to the correct device
        input_features = input_features.to(device)
        
        # Generate transcription using the fine-tuned model
        with torch.no_grad():
            predicted_ids = model.generate(input_features)
        
        # Decode the generated ids into text and append to the transcription
        transcription += processor.batch_decode(predicted_ids, skip_special_tokens=True)[0] + " "
    
    return transcription.strip()

# Example: Transcribe an audio file
audio_file = "detteholderikke.wav"
transcription = transcribe_audio(audio_file)
#implement prettyprint
from pprint import pprint


pprint(transcription)


('Jeg mener det er folkeskikk å ta imot en unnskyldning, jeg gjør det. Men '
 'dette holder ikke. Statsråden vittner ikke om å ha skjønt hva dette handler '
 'om. Hun sier at kommunikasjonen har rammet. Filmen har pisket opp '
 'stemninger. Og hun kan ikke fatte at folk reagerer sånn med å sende '
 'meldinger til Tarja Beck, eksempelvis. Det finnes hundrevis andre. Det er '
 'uroen, president, at ikke statsråden forstår at den typen melding som hun '
 'sier,')


In [5]:
from pprint import pprint


pprint(transcription)

('den jeg mener jeg faktisk ikke å ta imute nrunnskeldning jeg gjør det med '
 'dette holder ikke statsråden vi å sende mellinger til Terje bek eksempelvis '
 'til Fines Sunder vi seg Andre og president president at ikke statsråden '
 'første år at den typen mellninger til Terje bek eksempelvis til Fines Sunder '
 'vi seg Andre og president president president ikke statsråden første år at '
 'den typen typen mellning som hun sier ikke statsråden ikke statsråden første '
 'år at den typen typen typen mellning som hun sier ikke ikke statsråden ikke '
 'statsråden ikke statsråden ikke statsråden ikke statsråden ikke statsråden '
 'ikke statsråden ikke statsråden ikke statsråden ikke statsråden ikke '
 'statsråden ikke statsråden ikke statsråden ikke statsråden ikke statsråden '
 'ikke stats den typen typen typen typen typen typen mellning som hun sier '
 'ikke å den typ en typ en typ en typ en typ en typ en typ en typ en typ en '
 'typ en typ en typ en typ de ikke ikke ikke ikke ikke ikke